<a href="https://colab.research.google.com/github/riddhi-2080/Kaggle-Assignment/blob/master/ImageRec02_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Check for the currently available GPUs.

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9274319817941366915, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15319627182836158235
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13526977471818251281
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14648715520
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8389839544675827669
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [ ]:
# Import the basic libraries

import pandas as pd
import numpy as np

#### 2. View the files on current directory path

In [ ]:
!ls

anscombe.json		      mnist_test.csv	     sampleSubmission.csv
california_housing_test.csv   mnist_train_small.csv  trainLabels.csv
california_housing_train.csv  README.md


In [ ]:
#!rm test.7z


#### 3. Installing the Keras pre-processing

In [ ]:
!pip uninstall keras-preprocessing

In [ ]:
pip install git+https://github.com/keras-team/keras-preprocessing.git

  Cloning https://github.com/keras-team/keras-preprocessing.git to /tmp/pip-req-build-yk21t_sq
  Running command git clone -q https://github.com/keras-team/keras-preprocessing.git /tmp/pip-req-build-yk21t_sq
  Created wheel for Keras-Preprocessing: filename=Keras_Preprocessing-1.1.2-cp36-none-any.whl size=42983 sha256=5a6942586525d4a3140bf258d5992cab0ae205813cf840d21cdbf6bd29bab3e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1xewazx/wheels/03/a0/39/171f6040d36f36c71168dc69afa81334351b20955dc36ce932
Successfully built Keras-Preprocessing


#### 4. Import local files to google colab

In [ ]:
from google.colab import files

In [ ]:
# Mounting the drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Upload the actual data files from local system

uploaded = files.upload()

MessageError: ignored

In [ ]:
from keras.models import Sequential

#Import from keras_preprocessing 
from keras.layers import Input, Lambda, Dense, Activation, Flatten
from keras.models import Model

from keras.applications.vgg16 import VGG16, preprocess_input

from keras.preprocessing import image
from keras.preprocessing.image import .vgg16 import VGG16, preprocess_input

from keras.preprocessing.image import ImageDataGenerator

#from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

Using TensorFlow backend.


#### 5. Reading the Train Data csv file

In [ ]:
trainDataFrame=pd.read_csv("trainLabels.csv",dtype=str)

In [ ]:
trainDataFrame.head()

In [ ]:
def extFileType(fn):
    return fn+".png"

In [ ]:
trainDataFrame["id"]=trainDataFrame["id"].apply(extFileType)

In [ ]:
trainDataFrame.head(8)

In [ ]:

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.30)  #70:30 Train-Validation split

In [ ]:
# Need to give Filepath here
#
#
#
#

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

In [ ]:
!ls "/content/drive/My Drive"

'Colab Notebooks'


In [ ]:
#   !mkdir -p trainDrive
#   !google-drive-ocamlfuse drive

#### 6. Train-Validation Split of the Train Data

In [ ]:
train_generator=datagen.flow_from_dataframe(
dataframe=trainDataFrame,
directory="./train/",
x_col="id",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(242, 242))

In [ ]:
valid_generator=datagen.flow_from_dataframe(
dataframe=trainDataFrame,
directory="./train/",
x_col="id",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))
test_datagen=ImageDataGenerator(rescale=1./255.)

Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 50000 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

#### 7. Transfer Learning Model

In [ ]:
modelVGG = VGG16(weights = 'imagenet',
                 include_top = False,
                 input_tensor = Input(shape = (32, 32, 3)))

In [ ]:
for eachLayer in modelVGG.layers:
  eachLayer.trainable = False
  

In [ ]:
modelVGG.summary

In [ ]:
layerFlat = Flatten()(modelVGG.output)

In [ ]:
classFlat = Dense(1024, activation = 'relu')(layerFlat)

In [ ]:
outputFlat = Dense(10, activation = 'softmax')(classFlat)

In [ ]:
model1 = Model(inputs = modelVGG, outputs = output)
model1.summary()

In [ ]:
model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### 8. Fitting the Model

In [ ]:
model1.fit_generator(generator=train_generator,
                     steps_per_epoch=STEP_SIZE_TRAIN,
                     validation_data=valid_generator,
                     validation_steps=STEP_SIZE_VALID,
                     epochs=5
)